In [1]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import librosa
import librosa.display
from tqdm import tqdm

EMOTIONS = {1:"neu",2:'cal',3:'hap',4:'sad',5:'ang',6:'fea',7:'dis',8:'sur'}
EMOTION_NUM = 8
TILTLEEMOTIONS={"neu":'中性','cal':'平静','hap':'快乐','sad':'悲伤','ang':'生气','fea':'害怕','dis':'厌恶','sur':'惊讶'}
SOURCE_PATH = "db/RAVDESS/"
SAMPLE_RATE = 48000
EPOCH=10
FILE_PATH=[]
DURATION=3
OFFSET=0.5

df = pd.DataFrame(columns = ['FILE','EMOTION',])
FILE_NAME=[]
EMOTIONLIST=[]
for director, _, file_names in os.walk(SOURCE_PATH):
    for file_name in file_names:
        file_path = os.path.join(director+'/',file_name)
        FILE_NAME.append(file_path)
        EMOTIONLIST.append(file_name.split('.')[0].split('-')[2])
df["FILE"]=FILE_NAME
df["EMOTION"]=EMOTIONLIST

print("共有{}行".format(df.shape[0]))
df.head(n=10)

共有1440行


,FILE,EMOTION
0,db/RAVDESS/Actor_01/03-01-01-01-01-01-01.wav,01
1,db/RAVDESS/Actor_01/03-01-01-01-01-02-01.wav,01
2,db/RAVDESS/Actor_01/03-01-01-01-02-01-01.wav,01
3,db/RAVDESS/Actor_01/03-01-01-01-02-02-01.wav,01
4,db/RAVDESS/Actor_01/03-01-02-01-01-01-01.wav,02
5,db/RAVDESS/Actor_01/03-01-02-01-01-02-01.wav,02
6,db/RAVDESS/Actor_01/03-01-02-01-02-01-01.wav,02
7,db/RAVDESS/Actor_01/03-01-02-01-02-02-01.wav,02
8,db/RAVDESS/Actor_01/03-01-02-02-01-01-01.wav,02
9,db/RAVDESS/Actor_01/03-01-02-02-01-02-01.wav,02


In [2]:
from sklearn.model_selection import train_test_split

train_index,test_eval_index=train_test_split(list(df.index),test_size=0.2,random_state=1)
test_index,eval_index=train_test_split(test_eval_index,test_size=0.5,random_state=1)

print("train_index前10个:\t",train_index[:10])
print("eval_index前10个:\t",eval_index[:10])
print("test_index前十个:\t",test_index[:10])
print('train_index/(train_index+test_eval_index)={}/({}+{})={}'.format(len(train_index),len(train_index),len(test_eval_index),len(train_index)/(len(train_index)+len(test_eval_index))))
print('eval_index/(test_index+eval_index)={}/({}+{})={}'.format(len(eval_index),len(test_index),len(eval_index),len(eval_index)/(len(eval_index)+len(test_index))))


train_index前10个:	 [135, 368, 750, 801, 1322, 1060, 613, 994, 527, 484]
eval_index前10个:	 [1092, 495, 75, 1398, 215, 1108, 1279, 223, 735, 1053]
test_index前十个:	 [65, 1093, 925, 1170, 536, 675, 1417, 921, 1081, 1383]
train_index/(train_index+test_eval_index)=1152/(1152+288)=0.8
eval_index/(test_index+eval_index)=144/(144+144)=0.5


In [46]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

def getMELspec(data,sr):
    # shape=(n_mels, t)
    mel_spec = librosa.feature.melspectrogram(y = data,
                                              sr = SAMPLE_RATE,
                                              n_fft=1024,      # length of the FFT window
                                              win_length=1024,
                                              window='hamming',
                                              hop_length=512,
                                              n_mels=128
                                             )
    mel_spec_db = librosa.power_to_db(mel_spec,ref=np.max)
    mfccs = librosa.feature.mfcc(S=mel_spec_db)
    return mel_spec_db,mfccs

y_data=np.array(df['EMOTION'])
y_data=y_data.astype(np.int32)
y_data=y_data-1

alpha = 0.97
def PreEmphsised(data):
    return np.append(data[0],data[1:] - alpha * data[:-1])

def sameLenData(data):
    smLenData = np.zeros(SAMPLE_RATE*DURATION)
    smLenData[0:len(data)]=data
    return smLenData


def get3DMel(data):#model\\model2_1.pt  65%
    shape=data.shape
    diff_1=np.zeros(shape)
    diff_2=np.zeros(shape)
    diff_1[:,:shape[1]-1] = np.diff(data,n=1)
    diff_2[:,:shape[1]-2] = np.diff(data,n=2)
    return diff_1,diff_2,[data,diff_1 ,diff_2]

In [47]:
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import torch

scaler = StandardScaler()
DURATION=3
OFFSET=0.5
class SpeechDataset(Dataset):
    def __init__(self,index,setName=None):
        self.x_data = []
        self.arg = np.zeros([1,144000])
        self.y_data = []
        with tqdm(total=len(index)+1,desc="数据加载") as loadbar:
            for i in index: # 写入1440个文件的数据
                data, sr = librosa.load(df['FILE'][i],sr=SAMPLE_RATE,duration=DURATION,offset=OFFSET)
                smLenData = sameLenData(data)
                # 预加重
                smLenData = PreEmphsised(smLenData)
                # 特征向量
                mel_data,mfcc_data = getMELspec(smLenData,sr)
                # 差分向量
                #_,_,mel_data = get3DMel(mel_data)
                self.x_data.append(list(mel_data))
                self.y_data.append(y_data[i])
                loadbar.update(1)
#             if setName=='train_index':
#                 self.x_data=np.concatenate([self.x_data,arg_data],axis=0)
#                 self.y_data=np.concatenate([self.y_data,arg_label],axis=0)
            self.x_data=np.expand_dims(self.x_data,1)
            shape=np.array(self.x_data).shape
            self.x_data = np.reshape(self.x_data,newshape=(shape[0],-1))
            if setName=='train_index':
                self.x_data = scaler.fit_transform(self.x_data)
            else:
                self.x_data = scaler.transform(self.x_data)
            self.x_data=np.reshape(self.x_data,newshape=shape)
            self.x_data=torch.from_numpy(np.array(self.x_data,dtype=np.float64))
            self.y_data=torch.from_numpy(np.array(self.y_data,dtype=np.float64))
  
            self.len = len(index)
            loadbar.update(1)
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len


In [48]:
BATCH_SIZE=32

# 训练集加载
train_dataset = SpeechDataset(train_index,'train_index')
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=BATCH_SIZE,
                         )
# 验证集加载
eval_dataset = SpeechDataset(eval_index)
eval_loader = DataLoader(eval_dataset,
                          shuffle=False,
                          batch_size=BATCH_SIZE,
                         )
# 测试集加载
test_dataset = SpeechDataset(test_index)
test_loader = DataLoader(test_dataset,
                          shuffle=False,
                          batch_size=BATCH_SIZE,
                         )


数据加载: 100%|██████████████████████████████████████████████████████████████████████| 145/145 [00:05<00:00, 26.76it/s]


In [49]:
print(train_dataset.x_data.shape)
print(eval_dataset.x_data.shape)
print(test_dataset.x_data.shape)

torch.Size([1152, 1, 128, 282])
torch.Size([144, 1, 128, 282])
torch.Size([144, 1, 128, 282])


In [50]:
print('x_data:',train_dataset.x_data.shape,'  y_data',train_dataset.y_data.shape)
#np.set_printoptions(threshold=np.inf)
print(np.array(train_dataset.x_data[0]))
print(np.array(eval_dataset.x_data[0]))
print(np.array(test_dataset.x_data[0]))

x_data: torch.Size([1152, 1, 128, 282])   y_data torch.Size([1152])
[[[-0.58843891 -0.61338555 -0.63425847 ... -0.58457336 -0.58315172
   -0.33678262]
  [-0.59222255 -0.61235829 -0.63166393 ... -0.58066501 -0.58044706
   -0.31902948]
  [-0.58206291 -0.57093248 -0.5773697  ... -0.5151262  -0.52844289
    0.22484045]
  ...
  [-0.94305082 -0.96979468 -0.98087489 ... -0.19978231 -0.26722642
   -0.26782547]
  [-0.9401644  -0.96514657 -0.98022014 ... -0.11236172 -0.47035072
   -0.51433339]
  [-0.9415079  -0.9697074  -0.97096984 ...  0.07870125 -0.21945483
   -0.31626985]]]
[[[2.32988463 0.67576053 0.45627989 ... 4.237882   4.37996498 4.30726635]
  [2.32128456 0.64670886 0.4652489  ... 4.15544425 4.30618069 4.28622743]
  [2.79653376 0.89162831 0.8847009  ... 2.2928606  2.24635118 1.83709676]
  ...
  [2.24771959 1.56340517 1.85968225 ... 2.24434805 2.35348039 2.48341226]
  [2.52276261 2.21718987 1.95051432 ... 2.37648753 2.73744946 2.47510374]
  [2.06415279 1.98915273 1.88178599 ... 2.46286816

In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
class attention(nn.Module):
    def __init__(self,en_hidden_dim,de_hidden_dim,direction):
        super().__init__()
        middle_size = de_hidden_dim
        # H与s0拼接出来为(batch_size,src_len,en_hidden_dim*direction+de_hidden_dim)
        # 需要转化维度。首先，a(batch_size,src_len),公式2得知v(batch_size,?),E(?,src_len)
        # 因此?可以是任意维度，此处取de_hidden_dim
        self.s_en2de=nn.Linear(en_hidden_dim*2,de_hidden_dim)
        self.attn=nn.Linear(en_hidden_dim*direction+de_hidden_dim,middle_size)
        self.v = nn.Linear(middle_size,1)
        
    def forward(self,gru_output,gru_hidden):       
        # H(batch_size,src_len,en_hidden_dim*direction)
        _,src_len,_=gru_output.shape
        # s0(batch_size,en_hidden_dim*direction)
        # 但是s0作为dcoder初始隐藏状态，应该是s0(batch_size,de_hidden_dim)
        # 并且为了与H拼接，需要加一个维度，变成s0(batch_size,src_len,de_hidden_dim)
        s = torch.cat((gru_hidden[:, -2,: ], gru_hidden[:, -1,:]),dim=1)  # (batch_size,en_hidden_dim*direction)
        s = self.s_en2de(s)  # (batch_size,de_hidden_dim)
        # s=torc.tanh(s)  
        s=s.unsqueeze(1).repeat(1,src_len,1) # (batch_size,src_len,de_hidden_dim)     
        self.attn_hidden = torch.tanh(self.attn(torch.cat((s,gru_output),dim=2)))     
        attention_weight = self.v(self.attn_hidden).squeeze(2) #(batch_size,1,src_len)
        
        return F.softmax(attention_weight,dim=1)   #A dimension along which Softmax will be computed (so every slice along dim will sum to 1

In [64]:
hidden_size,input_size,p=128,32,0.3
kernel_size,stride=4,4
directional = 2

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        """
         input dimension every epoch is [32,1,128,282]
        """
        # conv2d
        self.conv2d=nn.Sequential(# No 1
                                   nn.Conv2d(in_channels=1,out_channels=16,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(16)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
            
                                  # No 2
                                  ,nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(32)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
            
                                  # No 3
                                  ,nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(64)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=4,stride=4)
                                  ,nn.Dropout(p)
            
                                  # No 4
                                  ,nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,stride=1,padding=1)
                                  ,nn.BatchNorm2d(64)
                                  ,nn.ReLU()
                                  ,nn.MaxPool2d(kernel_size=2,stride=2)
                                  ,nn.Dropout(p)
        )
        
        # GRU
        self.mp = nn.MaxPool2d(kernel_size=kernel_size,stride=stride)
        self.gru = nn.GRU(input_size=input_size,hidden_size=hidden_size,bidirectional =bool(directional-1),batch_first=True)
        self.dp = nn.Dropout(p)
        
        self.attention = attention(hidden_size,hidden_size,directional)
        self.emo_linear=nn.Linear(hidden_size*directional+2048,EMOTION_NUM) 
       
        self.out_dropout = nn.Dropout(0)
        
    def forward(self,x):
        x=x.float()
        
        # conv
        #convx=x[:,1].unsqueeze(1)
        convx = self.conv2d(x)
        convx = torch.flatten(convx,1) 
        #print(convx.shape)
        
        #GRU
        x = self.mp(x)   # input x:(N,C,freq,time)
        x = torch.squeeze(x)
        x = x.permute(0,2,1)  # batch_size, time, freq
        #x = torch.cat([x[:,0],x[:,1],x[:,2]],dim=2)
        
        # gru_output(batch_size, sequence length, hidden_size*directional)
        # gru_h(batch_size,n_layer*directional,hidden_size)
        # gru_h=[for_1,back_1,for_2,back_2...] 
        
        gru_output,gru_h=self.gru(x) # gru_output(B, S, H*2), gru_h(n_layer*directional,B,H)
        gru_h=gru_h.permute(1,0,2)
        gru_output = self.dp(gru_output)
        attention_weight = self.attention(gru_output,gru_h) # (batch_size,1,src_len)
        
        # (batch_size,1,src_len)*(batch_size,src_len,hidden_size*directional)
        # (batch_size,1,hidden_size*directional)
        attention_weight = attention_weight.unsqueeze(1)
#         print(attention_weight.shape)
#         print(gru_output.shape)
        attention = torch.bmm(attention_weight,gru_output)
        attention = torch.squeeze(attention,1) #（batch_size,hidden_size*directional）
        
        # cat
        output = torch.cat([convx,attention],dim=1)
        #print(attention.shape)
        #print(output.shape)
        output = self.out_dropout(self.emo_linear(output))
        return output

In [65]:
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,weight_decay=1e-3,momentum=0.5)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

loss_set=[]
accuracy_train=[]
accuracy_validate=[]
def train(epoch):
    running_loss = 0.0
    total = 0
    correct = 0
    for batch_idx,(input,target) in enumerate(train_loader,0):
        input,target = input.to(device),target.to(device)
        model.train()
        optimizer.zero_grad()
        output=model(input)
        _,pre = torch.max(output.data,dim=1)
        correct += (pre==target).sum().item()
        total += target.size(0)
        target = target.long()
        loss = criterion(output,target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loss_set.append(loss.item())
        if batch_idx % 20 == 19:
            print('[%d,%5d] loss: %.5f, running_loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / min((batch_idx+1)*BATCH_SIZE,df.shape[0]), running_loss))
            running_loss = 0
    loss_set.append(running_loss)
    accuracy_train.append(correct / total)
    print('Accacy on train_loader set: %d %% [%d/%d]' % (100 * correct / total, correct, total))   
      
#total = 0
#correct = 0
def validate(loader):
    total = 0
    correct = 0
    for (input,target) in (eval_loader if loader=='eval_loader' else test_loader):
        model.eval()
        input,target  =input.to(device),target.to(device)
        output = model(input)
        _,pre = torch.max(output.data,dim=1)
        correct += (pre==target).sum().item()
        total += target.size(0)  
    currect_rate = correct / total
    print('Accacy on %s set: %d %% [%d/%d]' % (loader,100 * correct / total, correct, total)) 
    accuracy_validate.append(currect_rate)
    return currect_rate

cuda:0


In [66]:
model_path='model\\model4.pt' 
opti_path='model\\opti4.pt' 
loss_path='model\\loss4.txt'
accur_train_path='model\\accur_train4.txt'
accur_valid_path='model\\accur_valid4.txt'
def saveModel(model_path):
    torch.save(model.state_dict(),model_path)
    torch.save(optimizer.state_dict(),opti_path) 
#     print("Model's state_dict:")
#     for param_tensor in model.state_dict():
#         print(param_tensor, "\t", model.state_dict()[param_tensor].size())
        
def loadModel(model_path):
    model = Model()
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    return model

def saveData(loss,accuracy_train,accuracy_validate):
    np.savetxt(loss_path,np.array(loss),fmt='%0.8f')
    np.savetxt(accur_train_path,np.array(accuracy_train),fmt='%0.8f')
    np.savetxt(accur_valid_path,np.array(accuracy_validate),fmt='%0.8f')

In [67]:
if __name__ == '__main__':
    max_correct_rate=0.0
    for epoch in range(1500):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)
        

[1,   20] loss: 0.09294, running_loss: 59.482
Accacy on train_loader set: 23 % [271/1152]
Accacy on eval_loader set: 16 % [24/144]
[2,   20] loss: 0.05799, running_loss: 37.115
Accacy on train_loader set: 34 % [393/1152]
Accacy on eval_loader set: 16 % [24/144]
[3,   20] loss: 0.05238, running_loss: 33.522
Accacy on train_loader set: 38 % [443/1152]
Accacy on eval_loader set: 16 % [24/144]
[4,   20] loss: 0.05102, running_loss: 32.651
Accacy on train_loader set: 41 % [480/1152]
Accacy on eval_loader set: 22 % [32/144]
[5,   20] loss: 0.04530, running_loss: 28.990
Accacy on train_loader set: 45 % [522/1152]
Accacy on eval_loader set: 16 % [24/144]
[6,   20] loss: 0.04318, running_loss: 27.634
Accacy on train_loader set: 48 % [557/1152]
Accacy on eval_loader set: 25 % [37/144]
[7,   20] loss: 0.04230, running_loss: 27.073
Accacy on train_loader set: 49 % [567/1152]
Accacy on eval_loader set: 34 % [49/144]
[8,   20] loss: 0.04227, running_loss: 27.050
Accacy on train_loader set: 50 % [582

Accacy on train_loader set: 80 % [923/1152]
Accacy on eval_loader set: 49 % [71/144]
[64,   20] loss: 0.01647, running_loss: 10.543
Accacy on train_loader set: 81 % [939/1152]
Accacy on eval_loader set: 56 % [82/144]
[65,   20] loss: 0.01588, running_loss: 10.162
Accacy on train_loader set: 80 % [933/1152]
Accacy on eval_loader set: 57 % [83/144]
[66,   20] loss: 0.01464, running_loss: 9.368
Accacy on train_loader set: 82 % [954/1152]
Accacy on eval_loader set: 53 % [77/144]
[67,   20] loss: 0.01485, running_loss: 9.506
Accacy on train_loader set: 81 % [938/1152]
Accacy on eval_loader set: 56 % [81/144]
[68,   20] loss: 0.01623, running_loss: 10.388
Accacy on train_loader set: 80 % [930/1152]
Accacy on eval_loader set: 48 % [70/144]
[69,   20] loss: 0.01452, running_loss: 9.295
Accacy on train_loader set: 82 % [949/1152]
Accacy on eval_loader set: 51 % [74/144]
[70,   20] loss: 0.01497, running_loss: 9.579
Accacy on train_loader set: 82 % [951/1152]
Accacy on eval_loader set: 55 % [80/

Accacy on eval_loader set: 54 % [79/144]
[126,   20] loss: 0.00923, running_loss: 5.908
Accacy on train_loader set: 89 % [1032/1152]
Accacy on eval_loader set: 65 % [94/144]
[127,   20] loss: 0.00945, running_loss: 6.045
Accacy on train_loader set: 88 % [1023/1152]
Accacy on eval_loader set: 59 % [86/144]
[128,   20] loss: 0.00857, running_loss: 5.482
Accacy on train_loader set: 90 % [1038/1152]
Accacy on eval_loader set: 63 % [92/144]
[129,   20] loss: 0.00783, running_loss: 5.009
Accacy on train_loader set: 91 % [1050/1152]
Accacy on eval_loader set: 56 % [82/144]
[130,   20] loss: 0.00778, running_loss: 4.982
Accacy on train_loader set: 92 % [1063/1152]
Accacy on eval_loader set: 58 % [84/144]
[131,   20] loss: 0.00908, running_loss: 5.812
Accacy on train_loader set: 89 % [1030/1152]
Accacy on eval_loader set: 62 % [90/144]
[132,   20] loss: 0.00804, running_loss: 5.144
Accacy on train_loader set: 90 % [1040/1152]
Accacy on eval_loader set: 61 % [89/144]
[133,   20] loss: 0.00777, r

Accacy on train_loader set: 94 % [1087/1152]
Accacy on eval_loader set: 67 % [97/144]
[188,   20] loss: 0.00570, running_loss: 3.647
Accacy on train_loader set: 93 % [1079/1152]
Accacy on eval_loader set: 68 % [98/144]
[189,   20] loss: 0.00599, running_loss: 3.833
Accacy on train_loader set: 94 % [1083/1152]
Accacy on eval_loader set: 69 % [100/144]
[190,   20] loss: 0.00578, running_loss: 3.698
Accacy on train_loader set: 94 % [1089/1152]
Accacy on eval_loader set: 70 % [101/144]
[191,   20] loss: 0.00515, running_loss: 3.297
Accacy on train_loader set: 92 % [1065/1152]
Accacy on eval_loader set: 63 % [92/144]
[192,   20] loss: 0.00496, running_loss: 3.174
Accacy on train_loader set: 94 % [1089/1152]
Accacy on eval_loader set: 68 % [98/144]
[193,   20] loss: 0.00495, running_loss: 3.171
Accacy on train_loader set: 94 % [1090/1152]
Accacy on eval_loader set: 65 % [95/144]
[194,   20] loss: 0.00608, running_loss: 3.893
Accacy on train_loader set: 94 % [1084/1152]
Accacy on eval_loader 

[249,   20] loss: 0.00415, running_loss: 2.656
Accacy on train_loader set: 96 % [1109/1152]
Accacy on eval_loader set: 63 % [92/144]
[250,   20] loss: 0.00396, running_loss: 2.535
Accacy on train_loader set: 96 % [1112/1152]
Accacy on eval_loader set: 69 % [100/144]
[251,   20] loss: 0.00358, running_loss: 2.293
Accacy on train_loader set: 96 % [1111/1152]
Accacy on eval_loader set: 67 % [97/144]
[252,   20] loss: 0.00336, running_loss: 2.153
Accacy on train_loader set: 96 % [1117/1152]
Accacy on eval_loader set: 68 % [98/144]
[253,   20] loss: 0.00425, running_loss: 2.723
Accacy on train_loader set: 95 % [1102/1152]
Accacy on eval_loader set: 63 % [92/144]
[254,   20] loss: 0.00339, running_loss: 2.167
Accacy on train_loader set: 95 % [1101/1152]
Accacy on eval_loader set: 63 % [91/144]
[255,   20] loss: 0.00459, running_loss: 2.937
Accacy on train_loader set: 95 % [1104/1152]
Accacy on eval_loader set: 68 % [99/144]
[256,   20] loss: 0.00417, running_loss: 2.666
Accacy on train_loade

...

[1230,   20] loss: 0.00075, running_loss: 0.477
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 74 % [107/144]
[1231,   20] loss: 0.00054, running_loss: 0.343
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 73 % [106/144]
[1232,   20] loss: 0.00081, running_loss: 0.522
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [108/144]
[1233,   20] loss: 0.00066, running_loss: 0.424
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[1234,   20] loss: 0.00064, running_loss: 0.413
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 73 % [106/144]
[1235,   20] loss: 0.00074, running_loss: 0.476
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1236,   20] loss: 0.00079, running_loss: 0.504
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 70 % [102/144]
[1237,   20] loss: 0.00061, running_loss: 0.387
Accacy 

[1291,   20] loss: 0.00059, running_loss: 0.376
Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 72 % [105/144]
[1292,   20] loss: 0.00062, running_loss: 0.395
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[1293,   20] loss: 0.00076, running_loss: 0.487
Accacy on train_loader set: 99 % [1141/1152]
Accacy on eval_loader set: 76 % [110/144]
[1294,   20] loss: 0.00105, running_loss: 0.670
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 73 % [106/144]
[1295,   20] loss: 0.00063, running_loss: 0.402
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 71 % [103/144]
[1296,   20] loss: 0.00059, running_loss: 0.375
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[1297,   20] loss: 0.00066, running_loss: 0.421
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1298,   20] loss: 0.00057, running_loss: 0.365
Accacy

Accacy on eval_loader set: 76 % [110/144]
[1352,   20] loss: 0.00069, running_loss: 0.441
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[1353,   20] loss: 0.00070, running_loss: 0.451
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[1354,   20] loss: 0.00091, running_loss: 0.579
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 74 % [107/144]
[1355,   20] loss: 0.00108, running_loss: 0.693
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 73 % [106/144]
[1356,   20] loss: 0.00076, running_loss: 0.486
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 75 % [109/144]
[1357,   20] loss: 0.00057, running_loss: 0.362
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [105/144]
[1358,   20] loss: 0.00058, running_loss: 0.371
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 72 % [105/144]
[1359,   20] 

Accacy on eval_loader set: 73 % [106/144]
[1413,   20] loss: 0.00074, running_loss: 0.471
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 72 % [105/144]
[1414,   20] loss: 0.00059, running_loss: 0.378
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [108/144]
[1415,   20] loss: 0.00052, running_loss: 0.334
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 70 % [102/144]
[1416,   20] loss: 0.00055, running_loss: 0.354
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[1417,   20] loss: 0.00077, running_loss: 0.490
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 74 % [107/144]
[1418,   20] loss: 0.00077, running_loss: 0.496
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 71 % [103/144]
[1419,   20] loss: 0.00083, running_loss: 0.529
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [104/144]
[1420,   20] 

Accacy on eval_loader set: 72 % [105/144]
[1474,   20] loss: 0.00053, running_loss: 0.342
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 68 % [98/144]
[1475,   20] loss: 0.00074, running_loss: 0.473
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 74 % [107/144]
[1476,   20] loss: 0.00053, running_loss: 0.339
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1477,   20] loss: 0.00053, running_loss: 0.337
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 70 % [101/144]
[1478,   20] loss: 0.00051, running_loss: 0.327
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[1479,   20] loss: 0.00048, running_loss: 0.309
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 70 % [102/144]
[1480,   20] loss: 0.00084, running_loss: 0.540
Accacy on train_loader set: 99 % [1142/1152]
Accacy on eval_loader set: 75 % [108/144]
[1481,   20] l

In [68]:
max_correct_rate

0.7916666666666666

In [69]:
if __name__ == '__main__':
    
    for epoch in range(2000):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)

[1,   20] loss: 0.00077, running_loss: 0.493
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[2,   20] loss: 0.00063, running_loss: 0.404
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 72 % [105/144]
[3,   20] loss: 0.00078, running_loss: 0.499
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 76 % [110/144]
[4,   20] loss: 0.00059, running_loss: 0.375
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[5,   20] loss: 0.00053, running_loss: 0.342
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[6,   20] loss: 0.00064, running_loss: 0.407
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 73 % [106/144]
[7,   20] loss: 0.00074, running_loss: 0.473
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[8,   20] loss: 0.00036, running_loss: 0.233
Accacy on train_loader set: 99 

Accacy on eval_loader set: 74 % [107/144]
[63,   20] loss: 0.00056, running_loss: 0.359
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 74 % [107/144]
[64,   20] loss: 0.00052, running_loss: 0.335
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 72 % [104/144]
[65,   20] loss: 0.00056, running_loss: 0.358
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[66,   20] loss: 0.00068, running_loss: 0.438
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [104/144]
[67,   20] loss: 0.00045, running_loss: 0.285
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[68,   20] loss: 0.00046, running_loss: 0.293
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 72 % [105/144]
[69,   20] loss: 0.00050, running_loss: 0.323
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 74 % [107/144]
[70,   20] loss: 0.00052, r

Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[125,   20] loss: 0.00067, running_loss: 0.429
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[126,   20] loss: 0.00053, running_loss: 0.340
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 77 % [112/144]
[127,   20] loss: 0.00056, running_loss: 0.357
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[128,   20] loss: 0.00052, running_loss: 0.330
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[129,   20] loss: 0.00061, running_loss: 0.392
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[130,   20] loss: 0.00051, running_loss: 0.327
Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 76 % [110/144]
[131,   20] loss: 0.00063, running_loss: 0.404
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_l

Accacy on eval_loader set: 77 % [111/144]
[186,   20] loss: 0.00047, running_loss: 0.299
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 73 % [106/144]
[187,   20] loss: 0.00048, running_loss: 0.307
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[188,   20] loss: 0.00083, running_loss: 0.531
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 76 % [110/144]
[189,   20] loss: 0.00054, running_loss: 0.347
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 76 % [110/144]
[190,   20] loss: 0.00069, running_loss: 0.440
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 77 % [112/144]
[191,   20] loss: 0.00046, running_loss: 0.294
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 79 % [114/144]
[192,   20] loss: 0.00083, running_loss: 0.529
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 72 % [104/144]
[193,   20] loss: 0.

[247,   20] loss: 0.00072, running_loss: 0.461
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 71 % [103/144]
[248,   20] loss: 0.00065, running_loss: 0.414
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[249,   20] loss: 0.00066, running_loss: 0.420
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [105/144]
[250,   20] loss: 0.00047, running_loss: 0.303
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [109/144]
[251,   20] loss: 0.00060, running_loss: 0.383
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[252,   20] loss: 0.00074, running_loss: 0.476
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[253,   20] loss: 0.00064, running_loss: 0.411
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[254,   20] loss: 0.00050, running_loss: 0.318
Accacy on train

...

[1714,   20] loss: 0.00043, running_loss: 0.278
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 77 % [111/144]
[1715,   20] loss: 0.00044, running_loss: 0.279
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [108/144]
[1716,   20] loss: 0.00032, running_loss: 0.202
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 75 % [108/144]
[1717,   20] loss: 0.00069, running_loss: 0.440
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 76 % [110/144]
[1718,   20] loss: 0.00062, running_loss: 0.394
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[1719,   20] loss: 0.00058, running_loss: 0.368
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 71 % [103/144]
[1720,   20] loss: 0.00063, running_loss: 0.406
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[1721,   20] loss: 0.00046, running_loss: 0.295
Accacy 

Accacy on eval_loader set: 71 % [103/144]
[1775,   20] loss: 0.00076, running_loss: 0.486
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 73 % [106/144]
[1776,   20] loss: 0.00077, running_loss: 0.490
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 75 % [109/144]
[1777,   20] loss: 0.00053, running_loss: 0.341
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 71 % [103/144]
[1778,   20] loss: 0.00050, running_loss: 0.321
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 74 % [107/144]
[1779,   20] loss: 0.00048, running_loss: 0.309
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[1780,   20] loss: 0.00044, running_loss: 0.284
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 70 % [102/144]
[1781,   20] loss: 0.00057, running_loss: 0.367
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [105/144]
[1782,   20] 

Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 72 % [105/144]
[1836,   20] loss: 0.00047, running_loss: 0.298
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 74 % [107/144]
[1837,   20] loss: 0.00038, running_loss: 0.243
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 72 % [104/144]
[1838,   20] loss: 0.00083, running_loss: 0.529
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[1839,   20] loss: 0.00052, running_loss: 0.334
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1840,   20] loss: 0.00079, running_loss: 0.507
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[1841,   20] loss: 0.00078, running_loss: 0.498
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 69 % [100/144]
[1842,   20] loss: 0.00059, running_loss: 0.376
Accacy on train_loader set: 99 % [1150/1152]
Accacy on

Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 74 % [107/144]
[1897,   20] loss: 0.00038, running_loss: 0.240
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1898,   20] loss: 0.00059, running_loss: 0.376
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 70 % [101/144]
[1899,   20] loss: 0.00047, running_loss: 0.302
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 74 % [107/144]
[1900,   20] loss: 0.00052, running_loss: 0.333
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1901,   20] loss: 0.00062, running_loss: 0.395
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 76 % [110/144]
[1902,   20] loss: 0.00056, running_loss: 0.361
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [108/144]
[1903,   20] loss: 0.00043, running_loss: 0.276
Accacy on train_loader set: 99 % [1150/1152]
Accacy on 

Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[1958,   20] loss: 0.00035, running_loss: 0.221
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[1959,   20] loss: 0.00055, running_loss: 0.352
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 76 % [110/144]
[1960,   20] loss: 0.00044, running_loss: 0.285
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[1961,   20] loss: 0.00045, running_loss: 0.291
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [109/144]
[1962,   20] loss: 0.00070, running_loss: 0.449
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 76 % [110/144]
[1963,   20] loss: 0.00044, running_loss: 0.281
Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 74 % [107/144]
[1964,   20] loss: 0.00058, running_loss: 0.372
Accacy on train_loader set: 99 % [1145/1152]
Accacy on

In [71]:
if __name__ == '__main__':
    
    for epoch in range(2000):
        train(epoch)
        correct_rate=validate('eval_loader')
        if max_correct_rate < correct_rate:
            max_correct_rate = correct_rate
            saveModel(model_path)

[1,   20] loss: 0.00057, running_loss: 0.365
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [109/144]
[2,   20] loss: 0.00049, running_loss: 0.313
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [108/144]
[3,   20] loss: 0.00041, running_loss: 0.262
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 76 % [110/144]
[4,   20] loss: 0.00052, running_loss: 0.330
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 77 % [111/144]
[5,   20] loss: 0.00099, running_loss: 0.636
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 75 % [109/144]
[6,   20] loss: 0.00046, running_loss: 0.297
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 75 % [109/144]
[7,   20] loss: 0.00038, running_loss: 0.244
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[8,   20] loss: 0.00044, running_loss: 0.284
Accacy on train_loader set: 99 

Accacy on eval_loader set: 73 % [106/144]
[63,   20] loss: 0.00063, running_loss: 0.405
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[64,   20] loss: 0.00066, running_loss: 0.422
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 74 % [107/144]
[65,   20] loss: 0.00103, running_loss: 0.657
Accacy on train_loader set: 98 % [1139/1152]
Accacy on eval_loader set: 70 % [101/144]
[66,   20] loss: 0.00060, running_loss: 0.387
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 75 % [108/144]
[67,   20] loss: 0.00043, running_loss: 0.273
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 76 % [110/144]
[68,   20] loss: 0.00074, running_loss: 0.472
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 75 % [108/144]
[69,   20] loss: 0.00039, running_loss: 0.252
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 74 % [107/144]
[70,   20] loss: 0.00052, r

Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[125,   20] loss: 0.00105, running_loss: 0.670
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 77 % [111/144]
[126,   20] loss: 0.00052, running_loss: 0.335
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [104/144]
[127,   20] loss: 0.00056, running_loss: 0.359
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 75 % [108/144]
[128,   20] loss: 0.00050, running_loss: 0.319
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 76 % [110/144]
[129,   20] loss: 0.00045, running_loss: 0.287
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[130,   20] loss: 0.00040, running_loss: 0.255
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 75 % [109/144]
[131,   20] loss: 0.00042, running_loss: 0.266
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_lo

Accacy on eval_loader set: 74 % [107/144]
[186,   20] loss: 0.00040, running_loss: 0.259
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[187,   20] loss: 0.00036, running_loss: 0.233
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 75 % [109/144]
[188,   20] loss: 0.00057, running_loss: 0.363
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 74 % [107/144]
[189,   20] loss: 0.00044, running_loss: 0.284
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 74 % [107/144]
[190,   20] loss: 0.00039, running_loss: 0.247
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[191,   20] loss: 0.00048, running_loss: 0.306
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[192,   20] loss: 0.00091, running_loss: 0.585
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 73 % [106/144]
[193,   20] loss: 0.

[247,   20] loss: 0.00065, running_loss: 0.413
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[248,   20] loss: 0.00078, running_loss: 0.500
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 77 % [112/144]
[249,   20] loss: 0.00036, running_loss: 0.232
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[250,   20] loss: 0.00040, running_loss: 0.255
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 76 % [110/144]
[251,   20] loss: 0.00063, running_loss: 0.402
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 75 % [108/144]
[252,   20] loss: 0.00041, running_loss: 0.261
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 71 % [103/144]
[253,   20] loss: 0.00059, running_loss: 0.375
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [104/144]
[254,   20] loss: 0.00087, running_loss: 0.555
Accacy on train

...

Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 76 % [110/144]
[1714,   20] loss: 0.00044, running_loss: 0.284
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 76 % [110/144]
[1715,   20] loss: 0.00084, running_loss: 0.535
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 70 % [101/144]
[1716,   20] loss: 0.00029, running_loss: 0.186
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 70 % [102/144]
[1717,   20] loss: 0.00044, running_loss: 0.283
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 67 % [97/144]
[1718,   20] loss: 0.00066, running_loss: 0.425
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 68 % [99/144]
[1719,   20] loss: 0.00038, running_loss: 0.245
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 74 % [107/144]
[1720,   20] loss: 0.00091, running_loss: 0.581
Accacy on train_loader set: 99 % [1146/1152]
Accacy on ev

Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 70 % [102/144]
[1775,   20] loss: 0.00064, running_loss: 0.408
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 79 % [115/144]
[1776,   20] loss: 0.00036, running_loss: 0.230
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 77 % [111/144]
[1777,   20] loss: 0.00036, running_loss: 0.230
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 73 % [106/144]
[1778,   20] loss: 0.00046, running_loss: 0.297
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[1779,   20] loss: 0.00035, running_loss: 0.223
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[1780,   20] loss: 0.00047, running_loss: 0.299
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[1781,   20] loss: 0.00071, running_loss: 0.451
Accacy on train_loader set: 99 % [1147/1152]
Accacy on 

[1835,   20] loss: 0.00073, running_loss: 0.466
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [105/144]
[1836,   20] loss: 0.00084, running_loss: 0.537
Accacy on train_loader set: 99 % [1143/1152]
Accacy on eval_loader set: 75 % [109/144]
[1837,   20] loss: 0.00052, running_loss: 0.332
Accacy on train_loader set: 99 % [1144/1152]
Accacy on eval_loader set: 70 % [102/144]
[1838,   20] loss: 0.00057, running_loss: 0.366
Accacy on train_loader set: 99 % [1145/1152]
Accacy on eval_loader set: 70 % [101/144]
[1839,   20] loss: 0.00047, running_loss: 0.302
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 76 % [110/144]
[1840,   20] loss: 0.00045, running_loss: 0.290
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[1841,   20] loss: 0.00060, running_loss: 0.383
Accacy on train_loader set: 99 % [1148/1152]
Accacy on eval_loader set: 72 % [105/144]
[1842,   20] loss: 0.00100, running_loss: 0.638
Accacy 

Accacy on eval_loader set: 73 % [106/144]
[1896,   20] loss: 0.00060, running_loss: 0.385
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 73 % [106/144]
[1897,   20] loss: 0.00044, running_loss: 0.279
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 73 % [106/144]
[1898,   20] loss: 0.00052, running_loss: 0.334
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [105/144]
[1899,   20] loss: 0.00062, running_loss: 0.399
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 76 % [110/144]
[1900,   20] loss: 0.00045, running_loss: 0.288
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 68 % [99/144]
[1901,   20] loss: 0.00049, running_loss: 0.313
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 72 % [104/144]
[1902,   20] loss: 0.00040, running_loss: 0.256
Accacy on train_loader set: 99 % [1150/1152]
Accacy on eval_loader set: 72 % [104/144]
[1903,   20] l

Accacy on eval_loader set: 70 % [102/144]
[1957,   20] loss: 0.00038, running_loss: 0.245
Accacy on train_loader set: 100 % [1152/1152]
Accacy on eval_loader set: 77 % [112/144]
[1958,   20] loss: 0.00055, running_loss: 0.353
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 68 % [99/144]
[1959,   20] loss: 0.00045, running_loss: 0.288
Accacy on train_loader set: 99 % [1151/1152]
Accacy on eval_loader set: 69 % [100/144]
[1960,   20] loss: 0.00054, running_loss: 0.347
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 72 % [105/144]
[1961,   20] loss: 0.00060, running_loss: 0.384
Accacy on train_loader set: 99 % [1146/1152]
Accacy on eval_loader set: 74 % [107/144]
[1962,   20] loss: 0.00067, running_loss: 0.432
Accacy on train_loader set: 99 % [1149/1152]
Accacy on eval_loader set: 74 % [107/144]
[1963,   20] loss: 0.00044, running_loss: 0.279
Accacy on train_loader set: 99 % [1147/1152]
Accacy on eval_loader set: 72 % [104/144]
[1964,   20] 

In [72]:
max_correct_rate

0.8194444444444444

In [70]:
max_correct_rate

0.8055555555555556

In [73]:
saveData(loss_set,accuracy_train,accuracy_validate)